In [174]:
import numpy as np
import random
#[cases, hospitalization, death]
infection = [[1,1,1],#white
            [1.7,3.7,2.4],#American native
            [0.7,1,1],#Asian
            [1.1,2.9,1.9],#black
            [1.3,3.1,2.3]]#hispanic & latino
            
prob = 1/np.array(infection).T[0]
prob = 0.1*prob

#race ratio from https://www.visualcapitalist.com/visualizing-u-s-population-by-race/
#white, American native, asian, balck, hispanic & latino
rr = np.array([0.601, 0, 0.056, 0.122, 0.185])
rr[1] = 1-rr.sum()
#print(rr)

# 0-17, 18-44, 45-64, 65-74, 75+
dr = np.array([0.06, 3.9, 22.4, 24.9, 48.7])*0.01
#death rate from https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/
#print(dr)
class person:
    def __init__(self, age, race, covid, antibody):
        self.age = age
        self.race = race
        self.covid = covid
        self.days_infected = 0
        self.antibody = antibody
    def get_death_rate(self):
        if self.age <= 17:
            return dr[0]
        elif self.age <= 44:
            return dr[1]
        elif self.age <= 64:
            return dr[2]
        elif self.age <= 74:
            return dr[3]
        else: return dr[4]
        
pool = []  
population_age_curve = []
for i in range(0,100):
    if i<40:
        population_age_curve.append(100-i/2)
    else:
        population_age_curve.append(80-8*(i-40)/6)
population_age_curve = np.array(population_age_curve)/sum(population_age_curve)
sampleList = np.arange(0,100,1)


for i in range(0,5):
    #print(i)
    for j in range(0, int(1000*rr[i])):
        pool.append(person(random.choices(sampleList, weights=population_age_curve)[0], i,random.choices([True, False], weights = [0.1*infection[i][0], 1-0.1*infection[i][0]]), False))
#len(pool)
#pool[601].covid

In [175]:
def simulation(days, inffection_rate, pool):
    for i in range(days):

        for person in pool:
            if person.covid == [True]:
                for j in pool:
                    if j.race == person.race:
                        if j.covid == [False] and j.antibody == False:
                            j.covid = random.choices([False,True], weights = [1-inffection_rate, inffection_rate])
                    else:
                        if j.covid == [False] and j.antibody == False:
                            j.covid = random.choices([False,True], weights = [1-inffection_rate/2, inffection_rate/2])

def get_badness_score(pool):
    count = 0
    for person in pool:
        if person.covid == [True]:
            count = count + person.get_death_rate()
    return count

def duplicate(pool):
    dup = []
    for i in pool:
        dup.append(person(i.age, i.race, i.covid, i.antibody))
        
    return dup

def get_candidate(num, pool):
    cands = []
    while len(cands) < 5:
        cand = random.choices(pool)
        for i in cands:
            if cand == i:
                continue
        cands.append(cand[0])
    return cands

def display(candidates):
    races = ["White", "American Native", "Aisan", "Black", "hispanic & latino"]
    for i in range(len(candidates)):
        print("the number",i+1, "candidate's info:")
        print("age: ", candidates[i].age)
        print("race: ", races[candidates[i].race])
        print("covid: ", candidates[i].covid)
        print("antibody: ", candidates[i].antibody, "\n")

def AI_choose(days, transmission_efficiency, candidates):
    duplicate_pools = []
    for i in range(len(candidates)):
        temp1 = candidates[i].antibody
        candidates[i].antibody = True
        temp2 = candidates[i].covid
        candidates[i].covid = [False]
        duplicate_pools.append(duplicate(pool))
        candidates[i].antibody = temp1
        candidates[i].covid = temp2
    low = 1000
    index = 0
    for i in range(len(duplicate_pools)):
        simulation(days, transmission_efficiency, duplicate_pools[i])
        s = get_badness_score(duplicate_pools[i])
        if s < low:
            low = s
            index = i
        print("candidate", i+1, "'s simulated badness score is:", s)
    print("AI's choose:")
    races = ["White", "American Native", "Aisan", "Black", "hispanic & latino"]
    
    print("the number",index+1, "candidate's info:")
    print("age: ", candidates[index].age)
    print("race: ", races[candidates[index].race])
    print("covid: ", candidates[index].covid)
    print("antibody: ", candidates[index].antibody, "\n")
    return candidates[index]
    

get_badness_score(pool)


12.037800000000002

In [176]:
candidates = get_candidate(5, pool)
display(candidates)

the number 1 candidate's info:
age:  52
race:  Black
covid:  [False]
antibody:  False 

the number 2 candidate's info:
age:  51
race:  White
covid:  [False]
antibody:  False 

the number 3 candidate's info:
age:  65
race:  White
covid:  [False]
antibody:  False 

the number 4 candidate's info:
age:  15
race:  White
covid:  [False]
antibody:  False 

the number 5 candidate's info:
age:  46
race:  White
covid:  [False]
antibody:  False 



In [177]:
AI_choose(10, 0.0001, candidates)

candidate 1 's simulated badness score is: 20.410200000000003
candidate 2 's simulated badness score is: 22.410799999999995
candidate 3 's simulated badness score is: 18.8566
candidate 4 's simulated badness score is: 21.194
candidate 5 's simulated badness score is: 21.008800000000004
AI's choose:
the number 3 candidate's info:
age:  65
race:  White
covid:  [False]
antibody:  False 

